imports

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import requests

helper function to format response objects into pandas dataframes

In [2]:
def call_endpoint(url, max_level=3, include_new_player_attributes=False):
    '''
    takes: 
        - url (str): the API endpoint to call
        - max_level (int): level of json normalizing to apply
        - include_player_attributes (bool): whether to include player object attributes in the returned dataframe
    returns:
        - df (pd.DataFrame): a dataframe of the call response content
    '''
    resp = requests.get(url).json()
    data = pd.json_normalize(resp['data'], max_level=max_level)
    included = pd.json_normalize(resp['included'], max_level=max_level)
    if include_new_player_attributes:
        inc_cop = included[included['type'] == 'new_player'].copy().dropna(axis=1)
        data = pd.merge(data, inc_cop, how='left', left_on=['relationships.new_player.data.id','relationships.new_player.data.type'], right_on=['id','type'], suffixes=('', '_new_player'))
    return data

calling leagues endpoint

In [3]:
url = 'http://partner-api.prizepicks.com/leagues' # leagues that are live on our board
df = call_endpoint(url)
df.head(3)

,id,type,attributes.name,attributes.rank,attributes.active,attributes.created_at,attributes.updated_at,attributes.icon,attributes.image_url,attributes.projections_count,relationships.projection_filters.data
0,150,league,CFB2H,0,True,2020-09-23T10:34:52.470-04:00,2021-08-30T07:21:18.109-04:00,football,https://static.prizepicks.com/images/leagues/f...,0,[]
1,9,league,NFL,1,True,2018-07-19T13:26:11.622-04:00,2021-08-30T07:21:18.116-04:00,football,https://static.prizepicks.com/images/leagues/f...,34,"[{'id': '1', 'type': 'projection_filter'}, {'i..."
2,163,league,NFL_FUT,2,True,2021-04-27T16:55:31.826-04:00,2021-08-30T07:21:36.776-04:00,football,https://static.prizepicks.com/images/leagues/f...,284,"[{'id': '1', 'type': 'projection_filter'}, {'i..."


calling projections endpoint

In [4]:
url = 'https://partner-api.prizepicks.com/projections'
df = call_endpoint(url)
df.head(3)

,id,type,attributes.line_score,attributes.risk_exposure,attributes.created_at,attributes.updated_at,attributes.description,attributes.start_time,attributes.status,attributes.board_time,attributes.rank,attributes.is_promo,attributes.discount_percentage,attributes.end_time,attributes.tv_channel,attributes.starting_pitcher,attributes.stat_type,attributes.projection_type,relationships.league.data.id,relationships.league.data.type,relationships.position.data,relationships.duration.data,relationships.stat_type.data.id,relationships.stat_type.data.type,relationships.new_player.data.id,relationships.new_player.data.type
0,243281,projection,15.0,None,2021-08-28T22:31:31.321-04:00,2021-08-29T08:20:20.707-04:00,JST,2021-09-01T19:30:00.000-04:00,pre_game,2021-07-01T17:00:00.000-04:00,1,False,None,None,None,None,Fantasy Score,Fantasy Score,15,league,None,None,14,stat_type,2615,new_player
1,244191,projection,9.5,None,2021-08-30T09:31:09.908-04:00,2021-08-30T11:45:43.564-04:00,ARI,2021-08-30T21:40:00.000-04:00,pre_game,2021-08-30T09:44:00.000-04:00,1,False,None,None,None,None,Fantasy Score,Fantasy Score,2,league,None,None,14,stat_type,38328,new_player
2,244169,projection,40.0,None,2021-08-30T09:09:17.636-04:00,2021-08-30T11:47:14.288-04:00,WSH,2021-08-30T19:05:00.000-04:00,pre_game,2021-08-30T09:22:00.000-04:00,1,False,None,None,None,None,Fantasy Score,Fantasy Score,43,league,None,None,14,stat_type,4835,new_player


calling projections endpoint with filters

In [5]:
url = 'https://partner-api.prizepicks.com/projections?league_id=9&per_page=10&single_stat=true'
df = call_endpoint(url)
df.head(3)

,id,type,attributes.line_score,attributes.risk_exposure,attributes.created_at,attributes.updated_at,attributes.description,attributes.start_time,attributes.status,attributes.board_time,attributes.rank,attributes.is_promo,attributes.discount_percentage,attributes.end_time,attributes.tv_channel,attributes.starting_pitcher,attributes.stat_type,attributes.projection_type,relationships.league.data.id,relationships.league.data.type,relationships.position.data,relationships.duration.data,relationships.stat_type.data.id,relationships.stat_type.data.type,relationships.new_player.data.id,relationships.new_player.data.type
0,238931,projection,315.5,None,2021-08-23T17:25:49.983-04:00,2021-08-23T20:01:50.836-04:00,CLE,2021-09-12T16:25:00.000-04:00,pre_game,2021-01-02T13:05:00.000-05:00,1,False,None,None,None,None,Passing Yards,Single Stat,9,league,None,None,16,stat_type,1093,new_player
1,224696,projection,23.5,None,2021-08-02T19:45:15.215-04:00,2021-08-02T23:59:26.262-04:00,NYJ,2021-09-12T13:00:00.000-04:00,pre_game,2021-01-02T13:05:00.000-05:00,1,False,None,None,None,None,Fantasy Score,Fantasy Score,9,league,None,None,14,stat_type,13106,new_player
2,224672,projection,80.5,None,2021-08-02T19:45:14.845-04:00,2021-08-02T20:09:21.411-04:00,NYJ,2021-09-12T13:00:00.000-04:00,pre_game,2021-01-02T13:05:00.000-05:00,1,False,None,None,None,None,Rushing Yards,Single Stat,9,league,None,None,15,stat_type,13106,new_player


calling projections endpoint with filters and including new_player level data (like name and team)

In [6]:
url = 'https://partner-api.prizepicks.com/projections?league_id=9&per_page=10&single_stat=true'
df = call_endpoint(url, include_new_player_attributes=True)
df.head(3)

,id,type,attributes.line_score,attributes.risk_exposure,attributes.created_at,attributes.updated_at,attributes.description,attributes.start_time,attributes.status,attributes.board_time,attributes.rank,attributes.is_promo,attributes.discount_percentage,attributes.end_time,attributes.tv_channel,attributes.starting_pitcher,attributes.stat_type,attributes.projection_type,relationships.league.data.id,relationships.league.data.type,relationships.position.data,relationships.duration.data,relationships.stat_type.data.id,relationships.stat_type.data.type,relationships.new_player.data.id,relationships.new_player.data.type,id_new_player,type_new_player,attributes.name,attributes.created_at_new_player,attributes.updated_at_new_player,attributes.image_url,attributes.position,attributes.team,attributes.fantasy_data_id,attributes.league_id,attributes.league,relationships.league.data.id_new_player,relationships.league.data.type_new_player
0,238931,projection,315.5,None,2021-08-23T17:25:49.983-04:00,2021-08-23T20:01:50.836-04:00,CLE,2021-09-12T16:25:00.000-04:00,pre_game,2021-01-02T13:05:00.000-05:00,1,False,None,None,None,None,Passing Yards,Single Stat,9,league,None,None,16,stat_type,1093,new_player,1093,new_player,Patrick Mahomes,2018-09-20T14:27:59.218-04:00,2020-07-30T18:28:24.765-04:00,https://s3-us-west-2.amazonaws.com/static.fant...,QB,KC,18890.0,9.0,NFL,9,league
1,224696,projection,23.5,None,2021-08-02T19:45:15.215-04:00,2021-08-02T23:59:26.262-04:00,NYJ,2021-09-12T13:00:00.000-04:00,pre_game,2021-01-02T13:05:00.000-05:00,1,False,None,None,None,None,Fantasy Score,Fantasy Score,9,league,None,None,14,stat_type,13106,new_player,13106,new_player,Christian McCaffrey,2020-01-21T12:24:33.593-05:00,2020-09-01T18:13:45.368-04:00,https://s3-us-west-2.amazonaws.com/static.fant...,RB,CAR,18877.0,9.0,NFL,9,league
2,224672,projection,80.5,None,2021-08-02T19:45:14.845-04:00,2021-08-02T20:09:21.411-04:00,NYJ,2021-09-12T13:00:00.000-04:00,pre_game,2021-01-02T13:05:00.000-05:00,1,False,None,None,None,None,Rushing Yards,Single Stat,9,league,None,None,15,stat_type,13106,new_player,13106,new_player,Christian McCaffrey,2020-01-21T12:24:33.593-05:00,2020-09-01T18:13:45.368-04:00,https://s3-us-west-2.amazonaws.com/static.fant...,RB,CAR,18877.0,9.0,NFL,9,league
